# ONTOP-Dremio Playground


**NOTE**: this notebook is written in JavaScript, so you need Jupyter + a JavaScript interpreter to run it. You may use the Deno interpreter, which is powerful and comes with a prebuild support for notebooks
```bash
curl -fsSL https://deno.land/install.sh | sh  # Install Deno
"$HOME/.deno/bin/deno" jupyter --install      # Add Deno to Jupyter
```
now just restart VSCode/Jupyter and you should see Deno amongst the kernels

In [1]:
const hello = "hello world!"
console.log(hello)

hello world!


In [8]:
const cmd = new Deno.Command("docker", { args: ["compose", "down"], stdout: "piped", stderr: "piped",}); 
await cmd.output();

{
  success: true,
  code: 0,
  signal: null,
  stdout: [Getter],
  stderr: [Getter]
}

In [9]:
// start docker compose if not already
const cmd = new Deno.Command("docker", { args: ["compose", "up", "-d"], stdout: "piped", stderr: "piped",}); 
const { success, stdout, stderr } = await cmd.output();
if (success) console.log("Docker Compose started successfully!");
else         console.error(new TextDecoder().decode(stderr));

Docker Compose started successfully!


## Setup

In [1]:
import * as u from '../utils/index.ts'

// Connect to postgres
const pg1 = await u.pgCreateClient(5531)
const pg2 = await u.pgCreateClient(5532)
const pg3 = await u.pgCreateClient(5533)

// Connect to dremio and add postgres sources if not already there
// IMPORTANT: if it's the first time running, it may fail. Just run it again
const drm = await u.drmCreateClient()
try {
  await Promise.all([
    u.drmSetupPostgres(drm,"postgres1", "postgres1"),
    u.drmSetupPostgres(drm, "postgres2", "postgres2"),
    u.drmSetupPostgres(drm, "postgres3", "postgres3"),
  ]);
}catch(e){
  if(e.httpStatus === 409) console.log("Postgres sources already there - status 409")
  else throw e
}


// Check if everything is up
console.log('')
await u.pgLogTables(pg1)
console.log(await u.drmApi(drm, 'GET', '/catalog'))

User alredy there, no need to create a new one
Auth token: diasdiu0nb2fpris8qbdlve99l - expires at: 22:04:00 (30hours)
Postgres sources already there - status 409

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
{
  data: [
    {
      id: "b7880582-c5c3-4be1-8fa5-0a275d83eb34",
      path: [ "@dremioUser" ],
      tag: "HE9xZ0EkRJc=",
      type: "CONTAINER",
      containerType: "HOME",
      createdAt: "2024-09-13T09:46:26.944Z"
    },
    {
      id: "0db052db-7a21-4e3e-ab9f-656abeae49fe",
      path: [ "postgres1" ],
      tag: "ttEIekt3CZI=",
      type: "CONTAINER",
      containerType: "SOURCE",
      createdAt: "2024-09-13T09:46:27.311Z"
    },
    {
      id: "c0dc2486-1e7c-4823-95b0-1850b1b903ec",
      path: [ "postgres3" ],
      tag: "PJRW3pTIcIo=",
      type: "CONTAINER",
      containerType: "SOURCE",
      createdAt: "2024-09-13T09:46:38.079Z"
    },
    {
      id: "2e9ca39e-66d1-45e3-9502-df967d342609",
      path: [ "postgres2" ],
      tag: "

In [2]:
u.settings.debug = false

await u.pgClear(pg1, pg2, pg3)
await u.pgLoadSQLData(pg1, './data/postgres/patients-serial.sql')
await u.pgLoadSQLData(pg1, './data/postgres/bloodtests-1.sql')
await u.pgLoadSQLData(pg2, './data/postgres/patients-uuid.sql')
await u.pgLogTables(pg1)
await u.pgLogTables(pg2)


console.log(await u.ontopRunQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://example.org/> 

    SELECT * WHERE {
      ?sub a :BloodTest;
           a ?type.
 	    FILTER(?type != :BloodTest)
    } 
    `))
    

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
Tables: [ { table_name: "patients" } ]
sub,type
http://example.org/bloodtest/1,http://example.org/Hemoglobin
http://example.org/bloodtest/2,http://example.org/Cholesterol
http://example.org/bloodtest/3,http://example.org/Hemoglobin
http://example.org/bloodtest/4,http://example.org/Cholesterol
http://example.org/bloodtest/5,http://example.org/Hemoglobin
http://example.org/bloodtest/6,http://example.org/Cholesterol
http://example.org/bloodtest/7,http://example.org/Hemoglobin
http://example.org/bloodtest/8,http://example.org/Cholesterol
http://example.org/bloodtest/9,http://example.org/Hemoglobin
http://example.org/bloodtest/10,http://example.org/Cholesterol
http://example.org/bloodtest/11,http://example.org/Hemoglobin
http://example.org/bloodtest/12,http://example.org/Cholesterol
http://example.org/bloodtest/13,http://example.org/Hemoglobin
http://example.org/bloodtest/14,http://example.org/Cholesterol
http://example.o

In [5]:

await u.pgClear(pg1, pg2, pg3)
await u.pgLoadSQLData(pg1, './data/postgres/patients-serial.sql')
await u.pgLoadSQLData(pg1, './data/postgres/bloodtests-2.sql')
await u.pgLoadSQLData(pg2, './data/postgres/patients-uuid.sql')

await u.pgLogTables(pg1)
await u.pgLogTables(pg2)


console.log(await u.ontopRunQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://example.org/> 

    SELECT * WHERE {
      ?sub a :BloodTest;
           a ?type.
 	    FILTER(?type != :BloodTest)
    } 
    `))
    

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
Tables: [ { table_name: "patients" } ]
sub,type
http://example.org/bloodtest/1,http://example.org/Hemoglobin
http://example.org/bloodtest/2,http://example.org/Cholesterol
http://example.org/bloodtest/3,http://example.org/White%20Blood%20Cell%20Count
http://example.org/bloodtest/4,http://example.org/Platelet%20Count
http://example.org/bloodtest/5,http://example.org/Glucose%20%28Fasting%29
http://example.org/bloodtest/6,http://example.org/Creatinine
http://example.org/bloodtest/7,http://example.org/Urea%20Nitrogen%20%28BUN%29
http://example.org/bloodtest/8,http://example.org/Sodium
http://example.org/bloodtest/9,http://example.org/Potassium
http://example.org/bloodtest/10,http://example.org/Calcium
http://example.org/bloodtest/11,http://example.org/Hemoglobin
http://example.org/bloodtest/12,http://example.org/Cholesterol
http://example.org/bloodtest/13,http://example.org/White%20Blood%20Cell%20Count
http://example.org/

In [24]:
for(let i = 0; i < 1000; i++){
    console.log(`(1, '2024-01-15', 'Hemoglobin${i+1}', 13.5, 'g/dL'),`)
}

(1, '2024-01-15', 'Hemoglobin1', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin2', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin3', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin4', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin5', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin6', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin7', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin8', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin9', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin10', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin11', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin12', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin13', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin14', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin15', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin16', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin17', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin18', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin19', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin20', 13.5, 'g/dL'),
(1, '2024-01-15', 'Hemoglobin

In [7]:

await u.pgClear(pg1, pg2, pg3)
await u.pgLoadSQLData(pg1, './data/postgres/patients-serial.sql')
await u.pgLoadSQLData(pg1, './data/postgres/bloodtests-big.sql')
await u.pgLoadSQLData(pg2, './data/postgres/patients-uuid.sql')
await u.pgLogTables(pg1)
await u.pgLogTables(pg2)


console.log(await u.ontopRunQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://example.org/> 

    SELECT * WHERE {
      ?sub a :BloodTest;
           a ?type.
 	    FILTER(?type != :BloodTest)
    } 
    `))
    

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
Tables: [ { table_name: "patients" } ]


NotFound: No such file or directory (os error 2): readfile './data/ontology/a41c3e80-71d2-11ef-b7db-135060252510-gitexclude.csv'